In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

from torch.autograd import Variable

import torch.utils.data

import pandas as pd
import numpy as np

import pdb

from tqdm import tqdm_notebook as tqdm

from torch.nn.utils import clip_grad_norm
import os

from decimal import Decimal

import seq2loc.models
import seq2loc.utils as utils
from seq2loc.datasets import SequenceDataset, PaddedSequenceDataset

from sklearn.metrics import precision_recall_fscore_support, average_precision_score

from tensorboardX import SummaryWriter

use_cuda = True

GPU_ids = [2]


os.environ['CUDA_VISIBLE_DEVICES'] = ','.join([str(ID) for ID in GPU_ids])
GPU_ids = list(range(0, len(GPU_ids)))
GPU_id = GPU_ids[0]

torch.cuda.device_count()

1

In [2]:


max_seq_len = 20000

GPU_id = 0
ds = PaddedSequenceDataset(SequenceDataset('./data/hpa_data_resized_train.csv', max_seq_len = max_seq_len, sequence_map='aa'), GPU_id = GPU_id)
mlb = ds.sequenceDataset.mlb
ds_validate = PaddedSequenceDataset(SequenceDataset('./data/hpa_data_resized_validate.csv', max_seq_len = max_seq_len, mlb = mlb, sequence_map='aa'), GPU_id = GPU_id)




/root/projects/seq2loc/seq2loc/datasets.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_sequences[nan_inds] = ''


In [3]:



#really big number to trim sequences to

# ds_test = PaddedSequenceDataset(SequenceDataset('./data/hpa_data_resized_test.csv', max_seq_len = max_seq_len, mlb = mlb, sequence_map='nucleotide'), GPU_id = GPU_id)

LR = 0.0001
N_EPOCHS = 500
hidden_size = 256
batch_size = 128
N_CLASSES = 32
N_LETTERS = len(ds.sequenceDataset.sequence_map)

criterion = torch.nn.BCEWithLogitsLoss()

enc = seq2loc.models.SeqConvResidClassifier(N_LETTERS, N_CLASSES, kernel_size = 8, layers_deep = 10, ch_intermed = 128, pooling_type='avg').cuda(GPU_id)

opt = optim.Adam(enc.parameters(), lr = LR)

writer = SummaryWriter()

save_dir = './classifier_conv_resid/{}'

iteration = 0

for epoch in range(N_EPOCHS):

    epoch_inds = utils.get_epoch_inds(len(ds), batch_size)
    pbar = tqdm(epoch_inds)

    epoch_losses = list()
    
    for batch in pbar:
        opt.zero_grad()

        x, y = ds[batch]        
        
        y_hat  = enc(x)
        
        loss = criterion(y_hat, y)
     
        loss.backward()
        opt.step()

        losses_np = np.squeeze(loss.detach().cpu().numpy())
        
        epoch_losses += [losses_np]
        pbar.set_description('%.4E' % Decimal(str(losses_np)))
        
        writer.add_scalars(save_dir.format('train'), {'loss': losses_np}, iteration)
        
        iteration += 1
    
    ###########################
    ### Write out test results
    ###########################
    enc.train(False)
    
    epoch_inds = utils.get_epoch_inds(len(ds_validate), batch_size)

    y_list = list()
    y_hat_list = list()
    losses_test = list()
    
    for batch in epoch_inds:

        x, y = ds_validate[batch]        

        with torch.no_grad():
            y_hat  = nn.Sigmoid()(enc(x))

        loss = criterion(y_hat, y)/y.shape[0]
        losses_test += [np.squeeze(loss.detach().cpu().numpy())]


        y_list += [y]
        y_hat_list += [y_hat]

    writer.add_scalars(save_dir.format('test'), {'loss': np.mean(losses_test)}, iteration)

#     y = torch.cat(y_list).cpu().numpy()
#     y_hat = torch.cat(y_hat_list).cpu().numpy()

#     thresh = 0.5


#     true_labs = y
#     pred_acts = y_hat
#     pred_labs = np.zeros_like(pred_acts)
#     pred_labs[pred_acts > thresh] = 1

#     df_stats = pd.DataFrame()
#     for i,col in enumerate(mlb.classes_):

#         # get true labels and predicted activations
#         true_labs_col = true_labs[:,i]
#         pred_acts_col = pred_acts[:,i]
#         pred_labs_col = pred_labs[:,i]

#         # compute one against all prec + recall stats
#         p,r,f,_ = precision_recall_fscore_support(true_labs_col,pred_labs_col, average='binary')

#         writer.add_scalars(save_dir.format('test_stats'), {'precision_{}'.format(col): p,
#                                                     'recall_{}'.format(col): r,
#                                                     'f1score_{}'.format(col): f,
#                                                     'auprc_{}'.format(col): average_precision_score(true_labs_col, pred_acts_col),
# #                                                     'support_{}'.format(col)] = int(true_labs_col.sum())
#                                                     }, iteration)


    enc.train(True) 

    pbar.set_description('%.4E' % Decimal(str(np.mean(epoch_losses))))
    


In [ ]:
ds = PaddedSequenceDataset(SequenceDataset('./data/hpa_data_resized_train.csv', max_seq_len = max_seq_len, sequence_map='nucleotide'), GPU_id = GPU_id)

In [39]:
x = Variable(torch.ones(1,1,105))
x[:,:,4:6] = 2



kernel_size = 5
stride = 2
padding = kernel_size/2

layer = torch.nn.Conv1d(1, 2, kernel_size = kernel_size, stride=stride, padding=padding)

transfer = torch.nn.AvgPool1d(kernel_size = stride, stride=stride, padding=0, ceil_mode=True, count_include_pad=False)

print(layer(x).shape)
print(transfer(x).shape)
print(x)
print(layer(x))


torch.Size([1, 2, 53])
torch.Size([1, 1, 53])
Variable containing:
( 0 ,.,.) = 

Columns 0 to 18 
    1   1   1   1   2   2   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 19 to 37 
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 38 to 56 
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 57 to 75 
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 76 to 94 
    1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1

Columns 95 to 104 
    1   1   1   1   1   1   1   1   1   1
[torch.FloatTensor of size (1,1,105)]

Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
  -0.5563 -0.8230 -1.9279 -1.5385 -1.0474 -1.0474 -1.0474 -1.0474 -1.0474
  0.0642  0.9562  0.7306  1.4305  0.7473  0.7473  0.7473  0.7473  0.7473

Columns 9 to 17 
  -1.0474 -1.0474 -1.0474 -1.0474 -1.0474 -1.0474 -1.0474 -1.0474 -1.0474
  0.7473  0.7473  0.7473  0.7473  0.7473  0.74

In [ ]:
N_LETTERS + 128

In [10]:
writer.file_writer.get_logdir()

'runs/May01_05-39-41_4d8c0372378e'

In [ ]:
df_stats

In [ ]:
y = torch.cat(y_list).cpu().numpy()
y_hat = torch.cat(y_hat_list).cpu().numpy()

for i in range(y.shape[1]):
    label_inds = np.where(y[:,i]>0)[0]
    
    print(np.mean(y[label_inds,i] == (y_hat[label_inds,i]>0.5)))


In [ ]:
(y_hat[label_inds,i]>0.5)

In [ ]:
import matplotlib.pyplot as plt

print(x.shape)
print()

plt.plot(losses)
plt.show()

In [ ]:
np.equal(y.cpu().data.numpy(), nn.Sigmoid()(y_hat).cpu().data.numpy()>0.5)

In [ ]:
np.log(np.exp(3.219125824868201)/28)

In [ ]:
enc.train(False)
dec.train(False)

x_tmp, _ = ds[[np.random.randint(len(ds))]]

# x = torch.unsqueeze(x[:,0,:],1)
batch_size_tmp = x_tmp.shape[1]

hidden = enc.initHidden(batch_size_tmp).cuda(GPU_id)
out, hidden = enc(x_tmp, hidden)

#input the stop character to the stream    
out = Variable(stopChar(batch_size_tmp)).cuda(GPU_id)


#     pdb.set_trace()
out_chars = list()

for i in range(x_tmp.shape[0]):

    out, hidden = dec(out, hidden) 
    
    out_chars += [tensorToChar(out)[0,0]]
    
enc.train(True)
dec.train(True)

print(''.join(np.hstack(tensorToChar(x_tmp))))
print(''.join(out_chars))

In [ ]:
3E-4*np.log(15)

In [ ]:
6.77e-05/np.log(15)